In [1]:
%pip install faker


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from faker import Faker
import random
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import polars as pl
import numpy as np
import pyarrow.parquet as pq
import uuid


In [3]:
df_profil = [
    {
        "Nom": "Fan de la nature",
        "pos": [161902431, 6256757776, 5771053254, 8731063849, 9609413254, 8615466894, 9356148774, 304351397, 304374138],
        "neg": [304850843, 9964615559, 6264550437, 2514192427, 11186120804, 9663434784]
    },
    {
        "Nom": "Historique (médieval)",
        "pos": [9964615559, 2080454089, 8309291920, 9767828163, 10815282268, 11006240930, 6635074831, 205161375, 312031207, 9964615559],
        "neg": [267878387, 6264550437, 6256757776, 304882303, 310235557]
    },
    {
        "Nom": "Monument typique (Grand lieu)",
        "pos": [6580307576, 249292417, 267878387],
        "neg": [8731063849, 8731063849, 6635074831, 9767828163]
    },
    {
        "Nom": "Historique (19,20ème)",
        "pos": [267878387, 6264550437, 2514258897, 2084125343, 407714718, 34050681, 271394429, 310235557, 368996291, 393969161],
        "neg": [312415707, 291231959, 9356148774, 11186120804, 6256757776, 34050692, 304374138]
    },
    {
        "Nom": "Artistique",
        "pos": [2514192427, 9663434784, 251316201, 11186120804, 291231959],
        "neg": [9356148774, 407714718, 11006240930, 9964615559, 310235557, 393969161]
    },
    {
        "Nom": "Antiquité",
        "pos": [4729709152, 874341418, 9864187809, 34050692],
        "neg": [267878387, 9609413254, 271394429, 6635074831, 6580307576]
    },
    {
        "Nom": "Fan de musée",
        "pos": [34050681, 393969161, 34050692, 160079640, 251316201, 252382829, 291231959, 311512162, 34050692],
        "neg": [9609413254, 6580307576, 2084125343, 407714718]
    },
    {
        "Nom": "Histoire (général)",
        "pos": [251316201, 252382829, 258816379, 291232006, 304882303, 310235557],
        "neg": [6256757776, 5771053254, 2514192427, 6580307576]
    },
    {
        "Nom": "Religieux",
        "pos": [251466390, 304850843, 307675986, 304850843],
        "neg": [251316201]
    }
]


In [4]:
df_profil

[{'Nom': 'Fan de la nature',
  'pos': [161902431,
   6256757776,
   5771053254,
   8731063849,
   9609413254,
   8615466894,
   9356148774,
   304351397,
   304374138],
  'neg': [304850843,
   9964615559,
   6264550437,
   2514192427,
   11186120804,
   9663434784]},
 {'Nom': 'Historique (médieval)',
  'pos': [9964615559,
   2080454089,
   8309291920,
   9767828163,
   10815282268,
   11006240930,
   6635074831,
   205161375,
   312031207,
   9964615559],
  'neg': [267878387, 6264550437, 6256757776, 304882303, 310235557]},
 {'Nom': 'Monument typique (Grand lieu)',
  'pos': [6580307576, 249292417, 267878387],
  'neg': [8731063849, 8731063849, 6635074831, 9767828163]},
 {'Nom': 'Historique (19,20ème)',
  'pos': [267878387,
   6264550437,
   2514258897,
   2084125343,
   407714718,
   34050681,
   271394429,
   310235557,
   368996291,
   393969161],
  'neg': [312415707,
   291231959,
   9356148774,
   11186120804,
   6256757776,
   34050692,
   304374138]},
 {'Nom': 'Artistique',
  'pos'

In [5]:
df_profil = pd.DataFrame(df_profil)

In [6]:
df_profil["Nom"]

0                 Fan de la nature
1            Historique (médieval)
2    Monument typique (Grand lieu)
3            Historique (19,20ème)
4                       Artistique
5                        Antiquité
6                     Fan de musée
7               Histoire (général)
8                        Religieux
Name: Nom, dtype: object

In [7]:
# Initialisation de Faker
fake = Faker()

nb_profil = 9
# Génération de 20 profils aléatoires avec noms et prénoms
noms = [fake.last_name() for _ in range(nb_profil)]
prenoms = [fake.first_name() for _ in range(nb_profil)]
profils_aleatoires = [df_profil['Nom'][i] for i in range(nb_profil)]
ids = [i for i in range(nb_profil)]

# Création du DataFrame
df_user = pd.DataFrame({
    "Id": ids,
    "Nom": noms,
    "Prénom": prenoms,
    "Profil": profils_aleatoires
})

# Ajout des points positifs (pos) et négatifs (neg) pour chaque profil
df_user["Pos"] = df_user["Profil"].apply(lambda x: df_profil[df_profil["Nom"] == x]["pos"].values[0])
df_user["Neg"] = df_user["Profil"].apply(lambda x: df_profil[df_profil["Nom"] == x]["neg"].values[0])

# Affichage du DataFrame
print(df_user)


   Id       Nom   Prénom                         Profil  \
0   0    Hudson    Tammy               Fan de la nature   
1   1  Thompson   Joshua          Historique (médieval)   
2   2    Prince  Timothy  Monument typique (Grand lieu)   
3   3      Cruz    Brian          Historique (19,20ème)   
4   4      Wood   Nicole                     Artistique   
5   5    Becker  Timothy                      Antiquité   
6   6   Simmons  Timothy                   Fan de musée   
7   7  Anderson     John             Histoire (général)   
8   8  Johnston     Ryan                      Religieux   

                                                 Pos  \
0  [161902431, 6256757776, 5771053254, 8731063849...   
1  [9964615559, 2080454089, 8309291920, 976782816...   
2                 [6580307576, 249292417, 267878387]   
3  [267878387, 6264550437, 2514258897, 2084125343...   
4  [2514192427, 9663434784, 251316201, 1118612080...   
5      [4729709152, 874341418, 9864187809, 34050692]   
6  [34050681, 393

In [8]:
# Ajout du user 10 (Quokka)
df_user = df_user.append({
    "Id": 10,
    "Nom": "Quokka",
    "Prénom": "Quokka",
    "Profil": "",  # Mettez le profil approprié pour Quokka
    "Pos": [26860223, 31425173],  # Mettez les points positifs appropriés pour Quokka
    "Neg": [31575884]  # Mettez les points négatifs appropriés pour Quokka
}, ignore_index=True)


C:\Users\HP\AppData\Local\Temp\ipykernel_5164\2556498607.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_user = df_user.append({


In [9]:
df_user

,Id,Nom,Prénom,Profil,Pos,Neg
0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
1,1,Thompson,Joshua,Historique (médieval),"[9964615559, 2080454089, 8309291920, 976782816...","[267878387, 6264550437, 6256757776, 304882303,..."
2,2,Prince,Timothy,Monument typique (Grand lieu),"[6580307576, 249292417, 267878387]","[8731063849, 8731063849, 6635074831, 9767828163]"
3,3,Cruz,Brian,"Historique (19,20ème)","[267878387, 6264550437, 2514258897, 2084125343...","[312415707, 291231959, 9356148774, 11186120804..."
4,4,Wood,Nicole,Artistique,"[2514192427, 9663434784, 251316201, 1118612080...","[9356148774, 407714718, 11006240930, 996461555..."
5,5,Becker,Timothy,Antiquité,"[4729709152, 874341418, 9864187809, 34050692]","[267878387, 9609413254, 271394429, 6635074831,..."
6,6,Simmons,Timothy,Fan de musée,"[34050681, 393969161, 34050692, 160079640, 251...","[9609413254, 6580307576, 2084125343, 407714718]"
7,7,Anderson,John,Histoire (général),"[251316201, 252382829, 258816379, 291232006, 3...","[6256757776, 5771053254, 2514192427, 6580307576]"
8,8,Johnston,Ryan,Religieux,"[251466390, 304850843, 307675986, 304850843]",[251316201]
9,10,Quokka,Quokka,,"[26860223, 31425173]",[31575884]


In [10]:

df_geo = pl.read_parquet("../data/transformed/poi_clean_category_geo.parquet")
df_poi_pl = df_geo.drop(["type", "geometry"])
df_poi = df_poi_pl.to_pandas()
df_poi.head()


,id,lat,lon,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,brand,brand:wikidata,...,description:covid19,payment:cash,payment:maestro,payment:mastercard,contact:mobile,url,contact:email,addr:suburb,sub_category,category
0,23656136,49.858471,6.364970,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,fitness_station,leisure
1,26860223,49.743478,6.089852,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,restaurant,amenity
2,30432808,50.856197,5.825697,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,attraction,tourism
3,31425173,50.844029,5.689079,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,city_gate,historic
4,31575884,50.844551,5.690282,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,library,amenity


In [11]:

def transform_to_user_item_interactions(df):
    interactions = []

    for _, row in df.iterrows():
        user = row["Nom"]
        profile = row["Profil"]
        id = row["Id"]

        # Ajouter les POI aimés comme interactions positives
        for poi_pos in row["Pos"]:
            pos_poi_info = df_poi[df_poi["id"] == poi_pos].index
            pos_poi_info = df_poi.iloc[pos_poi_info[0]]
            interactions.append((id, user, profile, pos_poi_info["id"],pos_poi_info["name"], pos_poi_info["sub_category"], pos_poi_info["category"], 1))

        # Ajouter les POI non aimés comme interactions négatives
        for poi_neg in row["Neg"]:
            neg_poi_info = df_poi[df_poi["id"] == poi_neg].index
            neg_poi_info = df_poi.iloc[neg_poi_info[0]]
            interactions.append((id, user, profile, neg_poi_info["id"], neg_poi_info["name"], neg_poi_info["sub_category"], neg_poi_info["category"], 0))

    return pd.DataFrame(interactions, columns=["Id", "User", "Profil", "POI-id", "POI-name", "POI-SubCat", "POI-Cat", "Liked (0 or 1)"])

# Exemple d'utilisation avec df_user
df_user_transformed = transform_to_user_item_interactions(df_user)

df_user_transformed


,Id,User,Profil,POI-id,POI-name,POI-SubCat,POI-Cat,Liked (0 or 1)
0,0,Hudson,Fan de la nature,161902431,point de vue en surplomb sur Bavigne et la nature,viewpoint,tourism,1
1,0,Hudson,Fan de la nature,6256757776,Fondry des Chiens,information,tourism,1
2,0,Hudson,Fan de la nature,5771053254,Cascade de Coo,viewpoint,tourism,1
3,0,Hudson,Fan de la nature,8731063849,Le Trou des Fées,attraction,tourism,1
4,0,Hudson,Fan de la nature,9609413254,vue en surplomb vers Oberpallen et la vallée d...,viewpoint,tourism,1
...,...,...,...,...,...,...,...,...
100,8,Johnston,Religieux,304850843,Chapelle Notre-Dame Auxiliatrice,place_of_worship,amenity,1
101,8,Johnston,Religieux,251316201,Musée Art & Histoire - Museum Kunst & Geschied...,museum,tourism,0
102,10,Quokka,,26860223,Restaurant Camping Krounebierg,restaurant,amenity,1
103,10,Quokka,,31425173,Nieuwenhofpoortje,city_gate,historic,1


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Select the rows corresponding to the target user (id 10)
target_user_row = df_user_transformed[df_user_transformed["Id"] == 10].iloc[1]

# Select the rows corresponding to other users
other_users_rows = df_user_transformed[df_user_transformed["Id"] != 10]

# Extract the interaction vectors for the target user and other users
target_user_vector = target_user_row.iloc[7:].values.reshape(1, -1)  # 7 corresponds to the index of the "Liked (0 or 1)" column
other_users_vectors = other_users_rows.iloc[:, 7:].values

# Compute cosine similarity
similarities = cosine_similarity(target_user_vector, other_users_vectors)

# Create a DataFrame to display the results
similarity_df = pd.DataFrame({
    "User": other_users_rows["User"].values,
    "Similarity": similarities[0]
})

# Sort the DataFrame by similarity in descending order
similarity_df = similarity_df.sort_values(by="Similarity", ascending=False)

# Display the result
#print(similarity_df)

# Merge similarity_df with df_user to get the details of similar users
similar_users_details = pd.merge(similarity_df, df_user, left_on="User", right_on="Nom", how="inner")

# Display the details of the most similar user
most_similar_user_details = similar_users_details.iloc[0]



In [13]:
pd.set_option('display.max_rows', 40)

similar_users_details.head(20)

,User,Similarity,Id,Nom,Prénom,Profil,Pos,Neg
0,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
1,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
2,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
3,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
4,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
5,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
6,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
7,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
8,Hudson,1.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."
9,Hudson,0.0,0,Hudson,Tammy,Fan de la nature,"[161902431, 6256757776, 5771053254, 8731063849...","[304850843, 9964615559, 6264550437, 2514192427..."


In [14]:
most_similar_user_details

User                                                     Hudson
Similarity                                                  1.0
Id                                                            0
Nom                                                      Hudson
Prénom                                                    Tammy
Profil                                         Fan de la nature
Pos           [161902431, 6256757776, 5771053254, 8731063849...
Neg           [304850843, 9964615559, 6264550437, 2514192427...
Name: 0, dtype: object

In [15]:
target_user_row

Id                               10
User                         Quokka
Profil                             
POI-id                     31425173
POI-name          Nieuwenhofpoortje
POI-SubCat                city_gate
POI-Cat                    historic
Liked (0 or 1)                    1
Name: 103, dtype: object